In [116]:
import matplotlib.pyplot as plt
import pickle
import json
import numpy as np
import math
from shapely.geometry import shape, Polygon
from PIL import Image, ImageDraw, ImageFont

In [91]:
with open('vaud-cities.geojson', 'r', encoding='utf-8') as f:
    geojson_vaud_cities = f.read()
vaud_cities = []
for city in json.loads(geojson_vaud_cities)['features']:
    vaud_cities.append({
        'name': city['properties']['name'],
        'tree_sparsity': city['properties']['tree_sparsity'],
        'tree_density': city['properties']['tree_density'],
        'geometry': city['geometry']
    })

In [92]:
sparsities = []
densities = []
for city in vaud_cities:
    sparsities.append(city['tree_sparsity'])
    densities.append(city['tree_density'])
    
sparsities = np.array(sparsities)
densities = np.array(densities)

sparsities = (sparsities - np.min(sparsities)) / np.ptp(sparsities)
densities = (densities - np.min(densities)) / np.ptp(densities)

arg_sorted_sparsities = np.argsort(sparsities)[::-1] # Inverse
arg_sorted_densities = np.argsort(densities)
    
reordered_sparsities = sparsities[arg_sorted_sparsities]
reordered_censities = densities[arg_sorted_densities]

In [93]:
def rgba_float_to_int(rgba):
    return tuple(int(round(c * 255)) for c in rgba)
def sparsity_to_color(sparsity):
    cmap = plt.cm.Greys
    return rgba_float_to_int(cmap(1-sparsity))
def density_to_color(density):
    cmap = plt.cm.Greens
    return rgba_float_to_int(cmap(density))

In [94]:
def get_y_range(geometry):
    data = shape(geometry)
    if isinstance(data, Polygon):
        polygons = [data]
    else:
        polygons = list(data)
    miny = np.inf
    maxy = 0
    all_ys = []
    data_points = {}
    for i, p in enumerate(polygons):
        x, y = p.exterior.xy
        data_points[i] = {}
        data_points[i]['y'] = np.array(y)
        cminy = data_points[i]['y'].min()
        cmaxy = data_points[i]['y'].max()
        if cminy < miny:
            miny = cminy
        if cmaxy > maxy:
            maxy = cmaxy
    return maxy - miny

In [101]:
def create_city_figure(geometry, density, y_range_min, y_ptp):
    data = shape(geometry)
    if isinstance(data, Polygon):
        polygons = [data]
    else:
        polygons = list(data)
    minx = np.inf
    miny = np.inf
    maxx = 0
    maxy = 0
    all_ys = []
    data_points = {}
    for i, p in enumerate(polygons):
        x, y = p.exterior.xy
        all_ys.extend(y)
        data_points[i] = {}
        data_points[i]['x'] = np.array(x)
        data_points[i]['y'] = np.array(y)
        cminx = data_points[i]['x'].min()
        cminy = data_points[i]['y'].min()
        cmaxx = data_points[i]['x'].max()
        cmaxy = data_points[i]['y'].max()
        if cminx < minx:
            minx = cminx
        if cminy < miny:
            miny = cminy
        if cmaxx > maxx:
            maxx = cmaxx
        if cmaxy > maxy:
            maxy = cmaxy
    averagey = np.mean(all_ys)
    xrange = maxx - minx # longitude
    yrange = maxy - miny # latitude

    xrange *= 111.320 / 110.574 * math.cos(math.radians(averagey))
    for i in list(data_points.keys()):
        data_points[i]['x'] *= 111.320 / 110.574 * math.cos(math.radians(averagey))
        cminx = data_points[i]['x'].min()
        if cminx < minx:
            minx = cminx

    aspect_ratio = xrange / yrange
    padding = 10
    normalized_y_range = (yrange - y_range_min) / y_range_ptp
    drawable_height = int(round((512 / 1.2) * (normalized_y_range + 0.2)))
    img_height = drawable_height + 2 * padding
    drawable_width = int(round(aspect_ratio * drawable_height))
    img_width = drawable_width + 2 * padding

    poly = Image.new('RGBA', (img_width, img_height))
    back = Image.new('RGBA', (img_width, img_height), (255, 255, 255, 0))
    polygon_drawer = ImageDraw.Draw(poly)
    for i in list(data_points.keys()):
        x = data_points[i]['x']
        y = data_points[i]['y']
        x = x - minx
        y = y - miny
        x /= xrange
        y /= yrange
        x = np.rint(x * drawable_width) + padding
        y = np.rint(y * drawable_height) + padding
        polygon_drawer.polygon(list(zip(x, y)), fill=density_to_color(density), outline=(71, 82, 94, 255))
    back.paste(poly, mask=poly)
    back = back.transpose(Image.FLIP_TOP_BOTTOM)
    return back

In [119]:
def paste_img_on_canvas_with_color(img, size, color, name):
    back = Image.new('RGBA', size, color)
    top_left_corner = int(round((size[0] - img.size[0]) / 2)), int(round((size[1] - img.size[1]) / 2))
    back.paste(img, top_left_corner, img)
    drawer = ImageDraw.Draw(back)
    drawer.text((10, 10), name, fill='#19a301')
    return back

In [103]:
y_ranges = []
for idx, city in enumerate(vaud_cities):
    y_range = get_y_range(city['geometry'])
    y_ranges.append(y_range)

In [104]:
y_range_min = np.min(y_ranges)
y_range_ptp = np.ptp(y_ranges)

In [105]:
for idx, city in enumerate(vaud_cities):
    img = create_city_figure(city['geometry'], densities[idx], y_range_min, y_range_ptp)
    img.save('FinalPoster/city-{}.png'.format(idx))

In [106]:
widths = []
heights = []
for idx, city in enumerate(vaud_cities):
    img = Image.open('FinalPoster/city-{}.png'.format(idx))
    w, h = img.size
    widths.append(w)
    heights.append(h)
widths = np.array(widths)
heights = np.array(heights)
print(np.max(widths))
print(np.max(heights))

566
532


In [120]:
for idx, city in enumerate(vaud_cities):
    img = Image.open('FinalPoster/city-{}.png'.format(idx))
    new_img = paste_img_on_canvas_with_color(img, (566, 566), sparsity_to_color(sparsities[idx]), city['name'])
    new_img.save('FinalPoster/new-city-{}.png'.format(idx))

In [108]:
x_dim = 28
y_dim = 11
idx_matrix = np.empty((y_dim, x_dim), dtype=np.int32)
for i in range(y_dim):
    c_arg_sparsities = arg_sorted_sparsities[i*x_dim:(i+1)*x_dim]
    c_arg_densities = []
    for arg_spars in c_arg_sparsities:
        c_arg_densities.append(np.where(arg_sorted_densities == arg_spars)[0][0])
    c_sorted_arg_densities = np.argsort(c_arg_densities)
    for j in range(x_dim):
        idx = c_arg_sparsities[c_sorted_arg_densities[j]]
        idx_matrix[i,j] = idx

In [121]:
# fig_size_a2 = (16.54, 23.39)
fig_size_a2 = (23.39, 16.54)

f, axarr = plt.subplots(y_dim, x_dim,
                        sharex='all',
                        sharey='all',
                        figsize=fig_size_a2,
                        tight_layout = {'pad': 0})
f.text(0.01, 0.01, 'Increasing Density →', ha='left')
f.text(0.01, 0.99, '← Decreasing Sparsity', va='top', rotation='vertical')
for y in range(y_dim):
    for x in range(x_dim):
        img = Image.open('FinalPoster/new-city-{}.png'.format(idx_matrix[y,x]))
        axarr[y,x].axis('off')
        axarr[y,x].imshow(img)
f.savefig('FinalPoster/output.pdf', dpi=300)
plt.close(f)